<a href="https://colab.research.google.com/github/ScrypticLabs/15.S60_2022/blob/main/Policy_Space_Method_Cartpole.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup 
The following code sets up requirements, imports, and helper functions. Please do not modify this section of code.

In [ ]:
!pip install gym[atari] > /dev/null 2>&1
!pip install pyvirtualdisplay > /dev/null 2>&1
!sudo apt-get update > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1

%matplotlib inline
import os
import math
import random
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.distributions import Categorical
import gym
from gym import spaces
from gym.wrappers import Monitor
import glob
import io
import base64
from IPython.display import HTML
from IPython import display as ipythondisplay
from tqdm.notebook import trange
import tqdm

In [ ]:
if 'setup_display' in locals():
    raise RuntimeError("Don't run this cell twice!")

setup_display = True
from pyvirtualdisplay import Display
display = Display(visible=False, size=(1400, 900))
display.start()

In [ ]:
"""
Utility functions to enable video recording of gym environment and displaying it.
To enable video, just do "env = wrap_env(env)
Adapt from https://colab.research.google.com/drive/1flu31ulJlgiRL1dnN2ir8wGh9p7Zij2t#scrollTo=8nj5sjsk15IT
"""
def show_video(directory='video'):
    mp4list = glob.glob(f'{directory}/*.mp4')
    if len(mp4list) > 0:
      mp4 = mp4list[0]
      video = io.open(mp4, 'r+b').read()
      encoded = base64.b64encode(video)
      ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                  loop controls style="height: 200px;">
                  <source src="data:video/mp4;base64,{0}" type="video/mp4" />
              </video>'''.format(encoded.decode('ascii'))))
    else: 
      print("Could not find video")
    

def wrap_env(env, directory='./video'):
    env = Monitor(env, directory, force=True)
    return env

In [ ]:
"""
Utility functions to load and save torch model checkpoints 
"""
def load_checkpoint(net, optimizer=None, step='max', save_dir='checkpoints'):
    os.makedirs(save_dir, exist_ok=True)

    checkpoints = [x for x in os.listdir(save_dir) if not x.startswith('events')]
    if step == 'max':
        step = 0
        if checkpoints:
            step, last_checkpoint = max([(int(x.split('.')[0]), x) for x in checkpoints])
    else:
        last_checkpoint = str(step) + '.pth'
    if step:
        save_path = os.path.join(save_dir, last_checkpoint)
        state = torch.load(save_path, map_location='cpu')
        net.load_state_dict(state['net'])
        if optimizer:
            optimizer.load_state_dict(state['optimizer'])
        print('Loaded checkpoint %s' % save_path)
    return step

def save_checkpoint(net, optimizer, step, save_dir='checkpoints'):
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    save_path = os.path.join(save_dir, str(step) + '.pth')

    torch.save(dict(net=net.state_dict(), optimizer=optimizer.state_dict()), save_path)
    print('Saved checkpoint %s' % save_path)

### Gym Environment
We will work with environment definitions from the OpenAI Gym library. Gym consists of several categories of environments (see https://gym.openai.com/ for some graphical examples):
- Classical control tasks: CartPole, MountainCar, etc. These tasks are relatively simple tasks which are sometimes also studied in control theory. They usually serve as diagnostic tasks for RL algorithms.
- Atari tasks: Breakout, Pong, SpaceInvader, etc. These tasks are based on games that ran on the [Atari System](https://en.wikipedia.org/wiki/Atari_2600). These tasks were popularized by DeepMind in the original DQN paper (linked above) and all have discrete action spaces, which is why a lot of DQN-based methods use these tasks as benchmarks.
- MuJoCo, Box2d, and other tasks. We will not discuss these tasks in this homework, but some are commonly used for DQN-based methods while other are used for policy-gradient-based methods.

We will work on the classical control task CartPole this time.


The below two cells demonstate interacting with Gym environments with uniformly randomly selected actions. Please familiarize with the method and attributes of the environment instance `env` that we use in these cells, in particular notice `env.reset()`, `env.step()`, `env.render()`, `env.observation_space`, and `env.action_space`.

In [ ]:
task= 'CartPole-v1'
video_dir = f'{task}_vis'
env = wrap_env(gym.make(task), video_dir)

observation = env.reset()

while True:
    env.render()
    
    # Sample a random action from the action space
    action = env.action_space.sample() 
    
    # Take a step in the environment with the action
    observation, reward, done, info = env.step(action) 
       
    if done:  # If the episode is done
      break
            
env.close()
show_video(video_dir)

Continuous spaces (such as observation space, which we called $\mathcal{S}$ in class) are defined by the `gym.Box` class; for example `env.observation_space` specifies the Numpy array shape (`env.observation_space.shape`) of each observation. Discrete spaces are defined by the `gym.Discrete` class; for example `env.action_space` represents the set `{0, 1, ..., env.action_space.n - 1}`, which we called $\mathcal{A}$ in class. 

In [ ]:
print('Observation (state) space:', env.observation_space)
print('Action space:', env.action_space)

Observation (state) space: Box(-3.4028234663852886e+38, 3.4028234663852886e+38, (4,), float32)
Action space: Discrete(2)


# Policy Gradient for Cartpole
We will implement the REINFORCE algorithm and some variants with PyTorch, a popular automatic differentiation framework. If you're not familiar with PyTorch, please find a tutorial online if necessary (e.g. [here](https://pytorch.org/tutorials/)).
<!-- 
**General comment: you do not need to strictly follow the method signature as defined in the starter code that we provide. We provide detailed types for the arguments and return values just as optional advice. You may modify the arguments as you see fit, as long as the overall structure is similar.** -->

We will be experimenting with

- Vanilla v.s. temporal structure
- No baseline v.s. a value function baseline

using a neural network with fully connected layers, which has input tensor size corresponding to the observation_space.shape, two intermediate layers each with size 64, and output size of action_space.n. As a reminder, the REINFORCE objective for the vanilla version is
$$V(\pi_\theta) = \mathbb{E}[\Big(\sum\limits_{t=0}^{T}\log\pi_\theta(a_t | s_t)\Big)\Big(\sum\limits_{t'=0}^{T} \gamma^{t'} r_{t'}\Big)].$$
The REINFORCE objective with temporal structure is
$$V(\pi_\theta) = \mathbb{E}[\sum\limits_{t=0}^{T}\log\pi_\theta(a_t | s_t)\Big(\sum\limits_{t'=t}^{T} \gamma^{t'} r_{t'}\Big)].$$

The REINFORCE objective with temporal structure and baseline is
$$V(\pi_\theta) = \mathbb{E}[\sum\limits_{t=0}^{T}\log\pi_\theta(a_t | s_t)\Big(\sum\limits_{t'=t}^{T} \gamma^{t'} r_{t'}-\gamma^t b(s_t)\Big)],$$
where $b(s_t)$ is the baseline.

## Policy and Value Network

**Fill out the Network class below for the policy and value functions. PGNetwork outputs a Categorical distribution over actions, and ValueNetwork outputs (estimated) value given the current observation. You can experiment with different architectures. It is strongly recommended to set "layer" (the number of hidden layers) as a parameter for PGNetwork, which will be quite useful in your subsequent experiments.**

In [ ]:
class PGNetwork(nn.Module):
    def __init__(self, in_dim, out_dim, hidden_dim = 64, layer = 2):
        """
        Initialize the parameter for the policy network
        """
        super(PGNetwork, self).__init__()
        # Note: here we are given the in_dim and out_dim to the network
        #### Your code here
    

    def forward(self, observation):
        """
        This function takes in a batch of observations and a batch of actions, and 
        computes a probability distribution (Categorical) over all (discrete) actions
        
        observation: shape (batch_size, observation_size) torch Tensor
        
        return: a categorical distribution over all possible actions. You may find torch.distributions.Categorical useful
        """
        #### Your code here



class ValueNetwork(nn.Module):
    def __init__(self, in_dim, hidden_dim = 64):
        """
        Initialize the parameter for the value function
        """
        super(ValueNetwork, self).__init__()
        #### Your code here
        

    def forward(self, observation):
        """
        This function takes in a batch of observations, and 
        computes the corresponding batch of values V(s)
        
        observation: shape (batch_size, observation_size) torch Tensor
        
        return: shape (batch_size,) values, i.e. V(observation)
        """
        #### Your code here


## Trajectory Rollout

**Fill out the following cells whenever you see `#### Your code here`.** Specifically, you need to complete the following functions
- `discounted_returns`, where you compute the cumulative discounted returns from the immediate rewards
- `update_parameters`, where you define the loss function and perform gradient update on the data collected from `rollout`.

In [ ]:
#### You do not need to change this cell 
def rollout(model, vmodel=None, device=None, MAX_T=10000, task=task):
    actions = torch.zeros(MAX_T, device=device, dtype=torch.int)
    rewards = torch.zeros(MAX_T, device=device)
    log_probs = torch.zeros(MAX_T, device=device)
    values = torch.zeros(MAX_T, device=device)
    obs = env.reset()
    T = 0
    ep_reward = 0
    while T < MAX_T:
        x = torch.tensor(obs, device=device, dtype=torch.float).unsqueeze(dim=0)
        dist = model(x)
        action = dist.sample()
        log_prob = dist.log_prob(action)
        if vmodel:
            value = vmodel(x)
            values[T] = value

        next_obs, reward, done, _ = env.step(action.item())
        ep_reward += reward
        rewards[T] = reward
        actions[T] = action
        log_probs[T] = log_prob
        obs = next_obs
        
        T += 1
        
        if done:
            break
    return actions[:T], rewards[:T], log_probs[:T], values[:T], ep_reward

In [ ]:
#### Compute the cumulative discounted return with discount factor gamma from the immediate rewards
def discounted_returns(rewards, gamma, device=None):
    returns = torch.zeros_like(rewards, device=device)
    #### Your code here
    
    return returns


#### Compute the policy loss from the returns and the log_probs
def update_parameters(optimizer, model, rewards, log_probs, gamma, 
                      values=None, temporal=False, device=None):
    # compute policy losses
    policy_loss = []
    returns = discounted_returns(rewards, gamma, device)
    eps = np.finfo(np.float32).eps.item()

    if values != None:
        #### Your code here: compute value loss by fitting values 
        #### to the returns with F.smooth_l1_loss
        value_loss = None
    
    if values != None:  # use the value function as the baseline
        returns = returns - values.detach()  # this is the "advantage"
        
    #### Your code here: compute policy loss based on different objectives
    if temporal:
        policy_loss = None

    if values != None:
        loss = policy_loss + value_loss
    else:
        loss = policy_loss
    
    # parameter update
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

# Experiments

Now we are ready to do the experiments. The following cell set up hyperparameters and construct two functions. `train()` trains the model. `visual()` visualizes the trained policy by creating a test environment and roll out a trajectory.

Note: You can play around with other configuration hyperparameters if you want.

In [ ]:
#### hyperparameters
device = torch.device('cpu') # change to 'cuda' when ready to run on GPU
gamma = 0.999
LR = 1e-3
task = 'CartPole-v1'
env = gym.make(task)
layer = 0
MAX_EPISODES = 1001
LOG_INTERVAL = 100
STEP_SAVE = 100 # Save the model in every STEP_SAVE steps
USE_VALUE_NETWORK = False
TEMPORAL = False


def train():
    #### Train the model
    model = PGNetwork(env.observation_space.shape[0], env.action_space.n, 64, layer).to(device)
    if USE_VALUE_NETWORK:
        vmodel = ValueNetwork(env.observation_space.shape[0], 64).to(device)
        optimizer = optim.Adam(list(model.parameters()) + list(vmodel.parameters()), lr=LR)
    else:
        optimizer = optim.Adam(model.parameters(), lr=LR)

    running_reward = 0
    history_reward = [] # store moving average of empirical rewards

    for step in trange(MAX_EPISODES):
        actions, rewards, log_probs, values, ep_reward = rollout(
            model, vmodel=vmodel if USE_VALUE_NETWORK else None, device=device, task=task)   
        running_reward = 0.05 * ep_reward + (1 - 0.05) * running_reward
        history_reward.append(running_reward)
        update_parameters(optimizer, model, rewards, log_probs, gamma, 
                          values=values if USE_VALUE_NETWORK else None, 
                          temporal=TEMPORAL, device=device)
        
        if step % LOG_INTERVAL == 0:
            print('Episode {}\tLast reward: {:.2f} \tAverage reward: {:.2f}'.format(
                  step, ep_reward, running_reward))
        
        if step % STEP_SAVE == 0:
            # Saves model checkpoint
            save_checkpoint(model, optimizer, step, save_dir=f'{task}_pg'+str(layer))
        
    save_checkpoint(model, optimizer, step, save_dir=f'{task}_pg'+str(layer))
    return history_reward


def visual(step='max'):
    #### visualize the learned policy by rolling out a trajectory

    task = 'CartPole-v1'
    video_dir = f'{task}_pg_vis'  # change the visualize direction if needed
    env_test = wrap_env(gym.make(task), video_dir)

    model = PGNetwork(env_test.observation_space.shape[0], env_test.action_space.n, 64, layer)  
    # change save_dir to the corresponding directory
    # You can adjust step
    _ = load_checkpoint(model, step=step, save_dir=f'{task}_pg'+str(layer))

    obs = env_test.reset()

    MAX_T = 5000
    sum_reward = 0
    for t in range(MAX_T):
        env.render()
        
        dist = model(torch.tensor(obs, dtype=torch.float).unsqueeze(dim=0))
        action = dist.logits.argmax().numpy()
        next_obs, reward, done, info = env_test.step(action)
        obs = next_obs
        sum_reward += reward
        
        if done:
            break

    print('%s steps' % t)
    print('%s reward' % sum_reward)
    env_test.close()
    show_video(video_dir)

## Vanilla v.s. Temporal

**(a): Print out the training process and the visualization results. Compare the performance between the two methods. What do you observe regarding the convergence and the rewards?**

*Enter you answer here.*

In [ ]:
#### Your code here
layer = 2
LR = 1e-3
MAX_EPISODES = 1001


## REINFORCE with Baseline

**(b): In the following, we always use the temporal structure. Try using REINFORCE with or without the baseline. Print out the training process and the visualization results. Plot the moving average of the rewards during the training process in a single figure (This is the output of train(). You can try multiple runs and take the mean). How does adding a baseline affect the convergence as well as the robustness of the training procedure?**

*Enter you answer here.*

In [ ]:
# Set parameters
TEMPORAL = True
layer = 2
LR = 1e-3
MAX_EPISODES = 1001

In [ ]:
#### Your code here: Run the test


In [ ]:
#### You code here: Plot the moving average rewards during the training process


## Hand-designed feature: Will it work?

On the OpenAI gym [leaderboard](https://github.com/openai/gym/wiki/Leaderboard#cartpole-v0), Zhiqing claimed to solve CartPole-v0 using an ingenious hand-designed policy. In particular, the policy is based on only the angle and the angular velocity (why this could work): if $3\theta + \dot\theta > 0$, take action $1$; otherwise take action $0$. Your task is to first implement this policy in the first cell, and then to run the second cell to visualize the results.

**(c): Describe the quality of this policy and discuss its potential limitations.**

*Enter your answer here.*

In [ ]:
# hand designed policy
def policy_naive(obs):
    #### Your code here
    return None

In [ ]:
#### visualize the learned policy by rolling out a trajectory
task = 'CartPole-v1'
video_dir = f'{task}_pg_vis'  # change the visualize direction if needed
env_test = wrap_env(gym.make(task), video_dir)

obs = env_test.reset()

MAX_T = 5000
sum_reward = 0
for t in range(MAX_T):
    env.render()
    
    action = policy_naive(obs)
    next_obs, reward, done, info = env_test.step(action)
    obs = next_obs
    sum_reward += reward
    
    if done:
        break

print('%s steps' % t)
print('%s reward' % sum_reward)
env_test.close()
show_video(video_dir)

You might be surprised at the performance of such a simple hand-designed policy. Such a policy can be viewed as a classic [PID controller](https://en.wikipedia.org/wiki/PID_controller) that makes the angle $\theta$ track $0$. Specifically, the policy is a thresholding PD controller. Now let us go one step further to gain some intuition about the performance of such a linear controller, and see if we can learn, by policy gradient, a policy that incorporates our intuition.

Consider an approximation of the system if the control $a$ is continuous and $\theta, \dot \theta$ are close to 0. Then, $\sin \theta \approx \theta$, $\cos \theta \approx 1$ and second-order terms $\theta^2, \dot \theta^2$ can be neglected if they appear together with constants or first-order terms $\theta, \dot \theta$. The transition ([source code](https://github.com/openai/gym/blob/master/gym/envs/classic_control/cartpole.py); [physical equations](https://coneural.org/florian/papers/05_cart_pole.pdf)), using $(x, \dot x, \theta, \dot \theta)$ as a state, is a linear dynamical system. To maximize the total reward, we want the cartpole to stay as vertical as possible; hence, we can redefine a cost function $c_1 \|\theta\|^2 + c_2 \|a\|^2$ (more generally, the first term is a quadratic function in $(x, \dot x, \theta, \dot \theta)$, if we want to further encourage it to stay around the center or to have low velocity). This falls into the LQ regime, to which the optimal solution is known to be a linear controller.

Since the system is indeed initialized at small $\theta, \dot \theta$, it makes sense to consider linear controllers like $\mu(s) = A s$. However, our action space is discrete in this problem, so we need to find a way to map the continuous actions to discrete ones, and such a mapping should be differentiable if we want to use policy gradient. One choice is to convert the value of the continuous action to a probability by the sigmoid function. Alternatively, you can use a $2 \times 4$ dimensional matrix $A$ to predict a $2$ dimensional vector, and then treat the components as logits (that is, to use softmax) to generate a discrete action.

**(d): Our intuition suggests such a "linear" policy could be reasonable. Now it's your turn to implement it, and to train it by REINFORCE. Specifically, you can play with different learning rates and different number of training episodes. What is the appropriate learning rate from your experiments? Report the linear coefficients from your policy and compare them with those from the hand-designed policy ($[0\ 0\ 3\ 1]^\top$). How do the number of parameters and the performance of your policy compared with those of REINFORCE with Baseline?**

*A: Enter your answer here.*

In [ ]:
#### Your code here

layer = 0
LR = 0
MAX_EPISODES = 0
USE_VALUE_NETWORK = None
